In [10]:
import pandas as pd
import numpy as np
import re
import cx_Oracle
import tqdm
from tqdm import tqdm

%run params.ipynb
cnxn = cx_Oracle.connect(obi_login, obi_pw, BD, encoding="UTF-8")   

def prError(s): print("\033[91m {}\033[00m" .format(s))

In [11]:
def Check_length(len1, len2):
    if (len1 != len2):
        prError("Ошибка соединения:")
        print(len1)
        print(len2)

##### Шаг 1. Подготовка вспомогательной таблицы, строки которой содержат каждый договор CONTRACT_KEY с датой срезов (все)

In [12]:
# all the tables with active portfolio of leasing cars on the cut dates from bogovich

# tables = list(['TABLE_31012017_2','TABLE_28022017_2','TABLE_31032017_2','TABLE_30042017_2',
#                'TABLE_31052017_2','TABLE_30062017_2','TABLE_31072017_2','TABLE_31082017_2','TABLE_30092017_2',
#                'TABLE_31102017_2','TABLE_30112017_2','TABLE_31122017_2','TABLE_31012018_2','TABLE_28022018_2',
#                'TABLE_31032018_2','TABLE_30042018_2','TABLE_31052018_2','TABLE_30062018_2','TABLE_31072018_2',
#                'TABLE_31082018_2','TABLE_30092018_2','TABLE_31102018_2','TABLE_30112018_2','TABLE_31122018_2',
#                'TABLE_31012019_2','TABLE_28022019_2','TABLE_31032019_2','TABLE_30042019_2','TABLE_31052019_2',
#                'TABLE_30062019_2','TABLE_31072019_2','TABLE_31082019_2','TABLE_30092019_2','TABLE_31102019_2',
#                'TABLE_30112019_2','TABLE_31122019_2','TABLE_31012020_2','TABLE_29022020_2','TABLE_31032020_2',
#                'TABLE_30042020_2','TABLE_31052020_2','TABLE_30062020_2','TABLE_31072020_2','TABLE_31082020_2',
#                'TABLE_30092020_2','TABLE_31102020_2','TABLE_30112020_2','TABLE_31122020_2','TABLE_31012021_2',
#                'TABLE_28022021_2','TABLE_31032021_2','TABLE_30042021_2','TABLE_31052021_2','TABLE_30062021_2',
#                'TABLE_31072021_2','TABLE_31082021_2','TABLE_30092021_2','TABLE_31102021_2','TABLE_30112021_2',
#                'TABLE_31122021_2'
# ])

tables = list(['TABLE_31012022_2', 'TABLE_28022022_2', 'TABLE_31032022_2', 'TABLE_30042022_2', 'TABLE_31052022_2', 'TABLE_30062022_2', 'TABLE_31072022_2'])
# tables = list(['TABLE_31012022_2']) # для тестирования

In [13]:
# all the dates Series
# Все срезы находятся в периоде  с 2017-2021 включительно, но поскольку необходимо рассчитать просрочку за последние 12 месяцев, то считаем за весь 2016 также

import calendar
from datetime import datetime

# yearstart = 2016
# yearend   = 2022

yearstart = 2021
yearend   = 2023 

Dates = pd.DataFrame([datetime(year, month, calendar.monthrange(year, month)[1]) 
                    for year in range(yearstart, yearend) for month in range(1, 13)])
Dates.columns = ['Date']
Dates.head(2)



,Date
0,2021-01-31
1,2021-02-28


In [14]:
# read all the CONTRACT_KEYs from all the table
# CONTRACT_KEY - первичный ключ для потрфеля, определяем для каких именно контрагентов нужна просрочка

keys = pd.DataFrame()

sql_query = "select DISTINCT CONTRACT_KEY,  SLICE_DT FROM YBOGOVICH.SSS"
for table in tqdm(tables, colour= 'green'):
    query = sql_query.replace('SSS', table);
    data = pd.read_sql(query, cnxn)
    keys = keys.append(data)  

contracts = pd.DataFrame(pd.unique(keys['CONTRACT_KEY']), columns=['CONTRACT_KEY'])
contracts.shape

100%|██████████| 7/7 [00:06<00:00,  1.04it/s]


(33380, 1)

In [15]:
# join all CONTRACT_KEYs with all the dates , 
# because the overdues will be calculated for all the cut dates to find the maximums for 3, 6, 12 months
# i - вспомогалтельный столбец для формирования таблицы для каждого 'CONTRACT_KEY' и каждой даты.
# можно было использовать функцию itertools.product()

contracts['i'] = 0
Dates['i'] = 0

Contracts_With_Dates= pd.merge(contracts, Dates, how = "outer" , on =['i'])[['CONTRACT_KEY', 'Date']]
Contracts_With_Dates.head()
Check_length(contracts.shape[0]*Dates.shape[0], Contracts_With_Dates.shape[0])

##### Шаг 2. подготовка исключаемых CONTRACT_KEY

Объяснение:
В таблице DM.DM_DETAILS_DAILY есть такие строки, которые показывают просрочку в разрезе PAYMENT_NUM. c флагом PAY_FLG = 0
однако платеж оплачен, так как за ним или перед идет  платеж с PAYMENT_NUM = 0. 
чтобы не усложнять алгоритм, и иметь возможность группировать по  PAYMENT_ITEM_KEY, такие договоры будут исключаться из расчета просрочки 
и показаны как имеющие 0 в просрочке. В противном случае они привносят ложно положительный результат

In [16]:
sql_query =   """SELECT DISTINCT CONTRACT_KEY
   FROM DM.DM_DETAILS_DAILY
  WHERE 
  PAYMENT_ITEM_KEY = 9
 AND PLAN_AMT IS  NULL """
excluded = pd.read_sql(sql_query, cnxn)
excluded.shape

(342, 1)

##### Шаг 3. Считаем Сумму текущей просрочки 

In [17]:
# считаем Сумму текущей просрочки на каждую дату среза для всех договоров DM.DM_DETAILS_DAILY
# лизинговые платежи имеют тип PAYMENT_ITEM_KEY = 9, именно по ним и будет считаться просрочка

from datetime import datetime
data_overdue = pd.DataFrame()

sql_query =   """ SELECT CONTRACT_KEY,
    :ДатаСреза ДатаСреза,
   SUM(DEBT) CURRENT_DEBT -- 	Сумма текущей просрочки на дату среза
FROM
  (SELECT CONTRACT_KEY,
    PAYMENT_ITEM_KEY,
    PAYMENT_NUM,
    MAX(DEBT) DEBT
  FROM
    (SELECT DM.DM_DETAILS_DAILY.*,
      FIRST_VALUE(PRE_PAY) OVER(PARTITION BY CONTRACT_KEY, PAYMENT_ITEM_KEY, PAYMENT_NUM, PLAN_PAY_DT_ORIG ORDER BY PAY_DT_ORIG) DEBT
    FROM DM.DM_DETAILS_DAILY
    WHERE PAYMENT_ITEM_KEY IN (  9)
    AND (PAY_DT_ORIG        >    :ДатаСреза   OR PAY_DT_ORIG       IS NULL  OR PAY_FLG            = 0)
     AND PLAN_PAY_DT_ORIG <=   :ДатаСреза    
    )
  GROUP BY CONTRACT_KEY,
    PAYMENT_ITEM_KEY,
    PAYMENT_NUM, PLAN_PAY_DT_ORIG 
  )
GROUP BY CONTRACT_KEY
  """
for index, row in tqdm(Dates.iterrows(), total=Dates.shape[0], colour= 'green'):
    data_overdue = data_overdue.append(pd.read_sql(sql_query, cnxn, params={'ДатаСреза':row['Date']}))
    # if index == 2: 
    #   break 
    
print(data_overdue.shape)
data_overdue.head(2)


100%|██████████| 24/24 [00:48<00:00,  2.03s/it]

(433611, 3)


,CONTRACT_KEY,ДАТАСРЕЗА,CURRENT_DEBT
0,1462,2021-01-31,7962378.62
1,1734,2021-01-31,1243914.98


In [18]:
# исключаем договора, с PAYMENT_NUM = 0 дающие ложноположительный результат
data_overdue = data_overdue[~data_overdue.CONTRACT_KEY.isin(excluded.CONTRACT_KEY)]
print(data_overdue.shape)

(433034, 3)


In [19]:
# соединяем с вспомогательной таблицей Contracts_With_Dates 
# получаем таблицу с Сумма текущей просрочки на дату среза Date по CONTRACT_KEY
contracts_overdue = pd.merge(Contracts_With_Dates, data_overdue, how='left', left_on=['CONTRACT_KEY', 'Date'], right_on=['CONTRACT_KEY', 'ДАТАСРЕЗА']).drop('ДАТАСРЕЗА', axis = 1)
Check_length(contracts_overdue.shape[0], Contracts_With_Dates.shape[0])
contracts_overdue.head(2)
# contracts_overdue[contracts_overdue['CURRENT_DEBT']>0]

,CONTRACT_KEY,Date,CURRENT_DEBT
0,706391,2021-01-31,NaN
1,706391,2021-02-28,NaN


In [20]:
# Расчитываем для каждой строки сумму просрочки за -1 (DEBT_-1), -2 (DEBT_-2), -3, -4 до -11(DEBT_-11) месяцев 
# и находим максимум за последние 3, 6 и 12 месяцев до даты среза 
contracts_overdue.sort_values(by = ['CONTRACT_KEY', 'Date'], ascending=[False, False], inplace=True)
for i in range(1, 12):
    col = 'DEBT_-'+str(i)
    contracts_overdue[col] = contracts_overdue.groupby(['CONTRACT_KEY'])['CURRENT_DEBT'].shift(-i)

contracts_overdue.fillna(0, inplace=True)

contracts_overdue["MAX_3"]  = contracts_overdue[["CURRENT_DEBT", "DEBT_-1", "DEBT_-2"]].max(axis=1)
contracts_overdue["MAX_6"]  = contracts_overdue[["CURRENT_DEBT", "DEBT_-1", "DEBT_-2", "DEBT_-3", "DEBT_-4", "DEBT_-5"]].max(axis=1)
contracts_overdue["MAX_12"] = contracts_overdue[["CURRENT_DEBT", "DEBT_-1", "DEBT_-2", "DEBT_-3", "DEBT_-4", "DEBT_-5", 
                                   "DEBT_-6", "DEBT_-7", "DEBT_-8", "DEBT_-9", "DEBT_-10", "DEBT_-11"]].max(axis=1)

print(contracts_overdue.shape)
contracts_overdue.head(2)


(801120, 17)


,CONTRACT_KEY,Date,CURRENT_DEBT,DEBT_-1,DEBT_-2,DEBT_-3,DEBT_-4,DEBT_-5,DEBT_-6,DEBT_-7,DEBT_-8,DEBT_-9,DEBT_-10,DEBT_-11,MAX_3,MAX_6,MAX_12
793559,1557242,2022-12-31,2138440.85,1710752.68,1283064.51,855376.34,427688.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2138440.85,2138440.85,2138440.85
793558,1557242,2022-11-30,1710752.68,1283064.51,855376.34,427688.17,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1710752.68,1710752.68,1710752.68


##### Шаг 4. считаем  максимальная длительность оплаченной просрочки и дата последней оплаченной просрочки

In [21]:
# считаем максимальная длительность оплаченной просрочки и дата последней оплаченной просрочки 
# на каждую дату среза для всех договоров DM.DM_DETAILS_DAILY

from datetime import datetime
last_overdue = pd.DataFrame()

sql_query =   """SELECT CONTRACT_KEY,
  :ДатаСреза ДатаСреза,
  MAX(PAY_DT_ORIG - PLAN_PAY_DT_ORIG) AS MAX_PAID_OVERDUE, -- максимальная длительность оплаченной просрочки
  MAX(PAY_DT_ORIG)                    AS LAST_PAID_DT_ORIG -- дата последней оплаченной просрочки
FROM  (
SELECT
    CONTRACT_KEY,
    PAYMENT_ITEM_KEY,  PAYMENT_NUM,
    MIN(PLAN_PAY_DT_ORIG) as PLAN_PAY_DT_ORIG, 
    CASE WHEN MAX(FLAG_NOTPAID) >0 THEN NULL ELSE MAX(PAY_DT_ORIG) END     AS PAY_DT_ORIG, 
    MAX(FLAG_NOTPAID) as FLAG_NOTPAID
    FROM
    (
SELECT CONTRACT_KEY,
    PAYMENT_NUM,
    PAYMENT_ITEM_KEY,
    PLAN_PAY_DT_ORIG AS PLAN_PAY_DT_ORIG,
    CASE WHEN PAY_DT_ORIG > &ДатаСреза THEN NULL ELSE PAY_DT_ORIG END as PAY_DT_ORIG,
    CASE WHEN PAY_DT_ORIG > &ДатаСреза THEN 1 WHEN PAY_DT_ORIG IS NULL THEN 1  ELSE 0 END as FLAG_NOTPAID
   FROM DM.DM_DETAILS_DAILY
  WHERE 
  PAYMENT_ITEM_KEY = 9
  AND (PAY_DT_ORIG > &ДатаСреза OR  PAY_DT_ORIG  IS NULL OR PAY_DT_ORIG > PLAN_PAY_DT_ORIG or PAY_FLG = 0)
  AND PLAN_PAY_DT_ORIG <= &ДатаСреза 
  AND PLAN_AMT IS NOT NULL
)
GROUP BY CONTRACT_KEY,PAYMENT_ITEM_KEY,  PAYMENT_NUM)
WHERE 
FLAG_NOTPAID = 0
GROUP BY CONTRACT_KEY"""

for index, row in tqdm(Dates.iterrows(), total=Dates.shape[0], colour= 'green'):
    last_overdue = last_overdue.append(pd.read_sql(sql_query, cnxn, params={'ДатаСреза':row['Date']}))
    # if index == 2: 
    #       break 
    
print(last_overdue.shape)
last_overdue.head(2)

100%|██████████| 24/24 [02:40<00:00,  6.69s/it]

(3366881, 4)


,CONTRACT_KEY,ДАТАСРЕЗА,MAX_PAID_OVERDUE,LAST_PAID_DT_ORIG
0,1316,2021-01-31,2,2017-06-26
1,1421,2021-01-31,86,2015-12-14


In [22]:
# исключаем договора, с PAYMENT_NUM = 0 дающие ложноположительный результат
last_overdue = last_overdue[~last_overdue.CONTRACT_KEY.isin(excluded.CONTRACT_KEY)]
print(last_overdue.shape)

(3361662, 4)


##### Шаг 5. считаем  Дата начала текущей просрочки и Количество дней просроченных платежей текущей задолженности

In [23]:
# считаем  дата начала текущей просрочки и количество дней просроченных платежей текущей задоленности 
# на каждую дату среза для всех договоров DM.DM_DETAILS_DAILY

from datetime import datetime
current_overdue = pd.DataFrame()

sql_query =   """SELECT CONTRACT_KEY, 
    :ДатаСреза ДатаСреза,
    MIN(PLAN_PAY_DT_ORIG)       AS StartOVERDUE, -- дата начала текущей просрочки
  &ДатаСреза-MIN(PLAN_PAY_DT_ORIG) AS OVERDUE -- количество дней просроченных платежей текущей задоленности
FROM  (
SELECT
    CONTRACT_KEY,
    PAYMENT_ITEM_KEY,  PAYMENT_NUM,
    MIN(PLAN_PAY_DT_ORIG) as PLAN_PAY_DT_ORIG, 
    CASE WHEN MAX(FLAG_NOTPAID) >0 THEN NULL ELSE MAX(PAY_DT_ORIG) END     AS PAY_DT_ORIG, 
    MAX(FLAG_NOTPAID) as FLAG_NOTPAID
    FROM
    (
SELECT CONTRACT_KEY,
    PAYMENT_NUM,
    PAYMENT_ITEM_KEY,
    PLAN_PAY_DT_ORIG AS PLAN_PAY_DT_ORIG,
    CASE WHEN PAY_DT_ORIG > &ДатаСреза THEN NULL ELSE PAY_DT_ORIG END as PAY_DT_ORIG,
    CASE WHEN PAY_DT_ORIG > &ДатаСреза THEN 1 WHEN PAY_DT_ORIG IS NULL THEN 1  ELSE 0 END as FLAG_NOTPAID
   FROM DM.DM_DETAILS_DAILY
  WHERE 
  PAYMENT_ITEM_KEY = 9
  AND (PAY_DT_ORIG > &ДатаСреза OR  PAY_DT_ORIG  IS NULL OR PAY_DT_ORIG > PLAN_PAY_DT_ORIG or PAY_FLG = 0)
  AND PLAN_PAY_DT_ORIG <= &ДатаСреза 
  AND PLAN_AMT IS NOT NULL
)
GROUP BY CONTRACT_KEY,PAYMENT_ITEM_KEY,  PAYMENT_NUM)
WHERE FLAG_NOTPAID = 1
GROUP BY CONTRACT_KEY"""

for index, row in tqdm(Dates.iterrows(), total=Dates.shape[0], colour= 'green'):
    current_overdue = current_overdue.append(pd.read_sql(sql_query, cnxn, params={'ДатаСреза':row['Date']}))
#     if index == 2: 
#          break 
    
print(current_overdue.shape)
current_overdue.head(2)

100%|██████████| 24/24 [01:13<00:00,  3.07s/it]

(433343, 4)


,CONTRACT_KEY,ДАТАСРЕЗА,STARTOVERDUE,OVERDUE
0,1462,2021-01-31,2016-07-10,1666
1,1734,2021-01-31,2016-08-03,1642


In [24]:
# исключаем договора, с PAYMENT_NUM = 0 дающие ложноположительный результат
current_overdue = current_overdue[~current_overdue.CONTRACT_KEY.isin(excluded.CONTRACT_KEY)]
print(current_overdue.shape)

(433034, 4)


##### Шаг 6.  Количество оплаченных лизинговых платежей

In [25]:
# считаем количество оплаченных лизинговый 
# отдельно по авансу WHEN PAYMENT_ITEM_KEY = 8 и лизинговым платежам PAYMENT_ITEM_KEY = 9

from datetime import datetime
paidnum_table = pd.DataFrame()

sql_query =   """SELECT CONTRACT_KEY,
  :ДатаСреза ДатаСреза,
  MAX(PAYMENT_NUM) AS LAST_PAYMENT_NUM -- количество оплаченных лизинговый платеж
FROM
  (SELECT CONTRACT_KEY,
    PAYMENT_NUM,
    MIN(AFTER_PAY) AS AFTER_PAY
  FROM
    (SELECT CONTRACT_KEY,
      PAYMENT_NUM,
      PAYMENT_ITEM_KEY,
      PLAN_PAY_DT_ORIG,
      PAY_DT_ORIG,
      PLAN_AMT,
      CASE WHEN PAYMENT_ITEM_KEY = 8 THEN FACT_PAY_AMT ELSE 0 END AS PrePay,
      CASE WHEN PAYMENT_ITEM_KEY = 9 THEN FACT_PAY_AMT ELSE 0 END AS LeasingPAY,
      AFTER_PAY
    FROM DM.DM_DETAILS_DAILY
    WHERE PAYMENT_ITEM_KEY IN (8, 9)
    AND PAY_DT_ORIG <= &ДатаСреза AND PAY_FLG = 1
    )
  WHERE PAYMENT_ITEM_KEY = 9
  GROUP BY CONTRACT_KEY, PAYMENT_NUM
  HAVING MIN(AFTER_PAY)<=0
  )
GROUP BY CONTRACT_KEY"""

for index, row in tqdm(Dates.iterrows(), total=Dates.shape[0], colour= 'green'):
    paidnum_table = paidnum_table.append(pd.read_sql(sql_query, cnxn, params={'ДатаСреза':row['Date']}))
    # if index == 2: 
    #      break 
    
print(paidnum_table.shape)
paidnum_table.head(2)

100%|██████████| 24/24 [03:13<00:00,  8.08s/it]

(5195184, 3)


,CONTRACT_KEY,ДАТАСРЕЗА,LAST_PAYMENT_NUM
0,1320,2021-01-31,48
1,1327,2021-01-31,32


##### Шаг 7.  Соединяем все рассчитанные поля с портфелем и записываем в файлы


In [26]:
# Проверка длины ИНН. Нужно чтоб было 12 цифр для физ лиц и 10 цифр для юр лиц
def Exam_INN(df):
    res = True
    inn = df[~df['INN'].str.len().isin([12, 10])]
    if inn.shape[0] != 0:
        print("ОШИБКА Срез {0}. {1} ошибок в длине ИНН. Например, CONTRACT_KEY = {2}".format(table, 
                                                    inn.shape[0], inn.iloc[0]['CONTRACT_KEY'], ))
        res = False
    return res

In [27]:
def initial_portfolio():
    return """select CONTRACT_KEY,
        SLICE_DT,
        STATUS_DT,
        STATUS_DESC,
        CONTRACT_NUM,
        OPEN_DT,
        CLOSE_DT,
        NEW_CLIENTNAME,
        NEW_CLIENT,
        NEW_CONTRACTID,
        NEW_LEASOBJECT,
        NEW_NUMBER_DKP,
        CARBREND,
        CARMODEL,
        QUOTEID,
        NEW_SROK,
        INN,
        OKVED,
        GROUPID,
        GROUPNAME,
        EQUIPMENT,
        DOP_IN_PRICE,
        COUNTRYIDNAME,
        CARCREATEYEAR,
        PURPOSE,
        SUPPLIERNAME,
        PRODUCTNAME,
        SUBPRODUCTNAME,
        OPPORTUNITYID,
        NEW_GROUP,
        NEW_PAIDTYPENAME,
        NEW_TOTALGRSUM,
        NEW_SUMWITHOUTADVPAYMENT,
        NEW_COUNTBYINGSUM,
        NEW_PLWITHOUTNDS,
        NEW_SUMRESULT,
        NEW_SUMMWITHDKP,
        NEW_PLNDOPDS,
        NEW_KO,
        NEW_AGENTFL,
        NEW_ADVPAYMENT
            FROM YBOGOVICH.SSS"""

In [28]:
def dict_rename_columns():
    return {'CONTRACT_KEY':'Ключ договора'
,'SLICE_DT':'Дата среза'
,'STATUS_DT':'Дата статуса самая ближайшая дате среза и меньше неё'
,'STATUS_DESC':'Статус на дату среза'
,'CONTRACT_NUM':'Номер договора лизинга'
,'OPEN_DT':'Дата договора лизинга'
,'CLOSE_DT':'Планируемая дата закрытия'
,'NEW_CLIENTNAME':'Наименование лизингополучателя'
,'NEW_CLIENT':'ID лизингополучателя'
,'NEW_CONTRACTID':'ID договора лизинга'
,'NEW_LEASOBJECT':'ID ТС'
,'NEW_NUMBER_DKP':'Договор купли-продажи'
,'CARBREND':'Марка ТС'
,'CARMODEL':'Модель ТС'
,'QUOTEID':'ID предложения'
,'NEW_SROK':'Срок лизинга'
,'INN':'ИНН'
,'OKVED':'Код ОКВЭД'
,'GROUPID':'ID группы компаний'
,'GROUPNAME':'Наименование группы компаний'
,'EQUIPMENT':'Наличие дополнительного оборудования'
,'DOP_IN_PRICE':'Дополнительное оборудование включено в стоимость'
,'COUNTRYIDNAME':'Страна марки'
,'CARCREATEYEAR':'Год выпуска ТС'
,'PURPOSE':'Цель приобретения ТС'
,'SUPPLIERNAME':'Наименование поставщика'
,'PRODUCTNAME':'Продукт'
,'SUBPRODUCTNAME':'Субпродукт'
,'OPPORTUNITYID':'ID лизинговой сделки'
,'NEW_GROUP':'Ликвидность ТС'
,'NEW_PAIDTYPENAME':'Вид лизинговых платежей '
,'NEW_TOTALGRSUM':'Общая сумма платежей по графику, в т.ч. НДС '
,'NEW_SUMWITHOUTADVPAYMENT':'Итого по графику без учета аванса, в т.ч. НДС '
,'NEW_COUNTBYINGSUM':'Выкупная стоимость в расчет, рубли '
,'NEW_PLWITHOUTNDS':'Стоимость ПЛ за вычетом НДС '
,'NEW_SUMRESULT':'Итоговая сумма с НДС '
,'NEW_SUMMWITHDKP':'Стоимость имущества за вычетом НДС в валюте ДКП'
,'NEW_PLNDOPDS':'Стоимость имущества к расчету по ДКП, в т.ч. НДС '
,'NEW_KO':'Сумма скидки '
,'NEW_AGENTFL':'Аванс ЛП, % '
,'NEW_ADVPAYMENT':'Сумма Авансового платежа, подлежащего уплате '
,'MAX_PAID_OVERDUE':'Максимальная длительность оплаченной просрочки'
,'LAST_PAID_DT_ORIG':'Дата последней оплаченной просрочки'
,'STARTOVERDUE':'Дата начала текущей просрочки'
,'OVERDUE':'Количество дней просроченных платежей текущей задоленности'
,'LAST_PAYMENT_NUM':'Количество оплаченных лизинговых платежей'
,'CURRENT_DEBT':'Сумма текущей задолженности'}

In [29]:
def FormatData(col):
    result[col] = result[col].astype(str)
    pattern_for_int = r'0001-01-01' #  пустая дата. заменяем ее на значение np.nan. Для дальнейшего перевода в формат даты
    result.loc[result[col].str.match(pattern_for_int, flags = re.I)==True, col] = np.nan
    result[col] = pd.to_datetime(result[col], format='%Y.%m.%d')

In [30]:
contracts_overdue = contracts_overdue[['CONTRACT_KEY', 'Date', 'CURRENT_DEBT', 'MAX_3', 'MAX_6', 'MAX_12']]

sql_query = initial_portfolio()
new_names = dict_rename_columns()

for table in tqdm(tables, colour= 'green'):
    query = sql_query.replace('SSS', table);
    data = pd.read_sql(query, cnxn)
    result = (pd.merge(data, last_overdue, how='left', left_on=['CONTRACT_KEY', 'SLICE_DT'], 
                      right_on=['CONTRACT_KEY', 'ДАТАСРЕЗА']).drop('ДАТАСРЕЗА', axis = 1).
              merge(current_overdue, how='left', left_on=['CONTRACT_KEY', 'SLICE_DT'], 
                      right_on=['CONTRACT_KEY', 'ДАТАСРЕЗА']).drop('ДАТАСРЕЗА', axis = 1).
              merge(paidnum_table, how='left', left_on=['CONTRACT_KEY', 'SLICE_DT'], 
                      right_on=['CONTRACT_KEY', 'ДАТАСРЕЗА']).drop('ДАТАСРЕЗА', axis = 1).
              merge(contracts_overdue, how='left', left_on=['CONTRACT_KEY', 'SLICE_DT'], 
                      right_on=['CONTRACT_KEY', 'Date']).drop('Date', axis = 1))

    Check_length(data.shape[0], result.shape[0])
    Exam_INN(result) # проверка длины ИНН
    FormatData('CLOSE_DT')  # меняем у данных формат со строки на дату



    name = "SUMMARY\\" + "portfolio_" + table[10:14] + table[8:10] + table[6:8]
    result.rename(columns = new_names, inplace = True)
#     result.to_csv(name+".csv", sep=";", index = False,  date_format='%d.%m.%Y', encoding="utf-8", decimal=',')
    result.to_csv(name+".csv", sep=";", index = False,  date_format='%Y.%m.%d', encoding="utf-8", decimal=',')
    

100%|██████████| 7/7 [00:43<00:00,  6.26s/it]


In [31]:
cnxn.close()

In [35]:
# result.info()

##### Комментарии

CONTRACT_KEY = 954294
PURPOSE = '... ПАО «Новатэ́к»...'
содержит символы, которые не могут быть записаны в кодировке "windows-1251", поэтому кодировка менятся на "utf-8"


In [33]:
def identify_duplicates(df, columns):
    """identifies duplicates in columns of DataFrame

    Args:
        df ([DataFrame]): DataFrame
        columns ([list]): list of columns

    Returns:
        num: [int] 
            number of duplicate rows 
        DataFrame  
            A DataFrame consisting of repeated rows in the specified columns
    """
 
    rows = df.groupby(columns).size().reset_index()
    rows.rename({0: 'row_nums'}, axis = 1, inplace = True)

    dupl = rows[rows['row_nums'] > 1] # дубликаты
    
    return len(dupl), pd.merge(df, dupl, how = 'inner', on = columns)

In [34]:
# # определяем дубликаты по CONTRACT_KEY в исходных таблицах
# sql_query = initial_portfolio()

# for table in tables:
#     query = sql_query.replace('SSS', table);
#     data = pd.read_sql(query, cnxn)
#     num, duplicates = identify_duplicates(data, ['CONTRACT_KEY'])
#     print('table = {0}, количество дубликатов {1}'.format(table, num)) 
    